In [1]:
import sys
sys.path.append('/u/trahoang/Desktop')
sys.path.append('/u/trahoang/Desktop/SummerIntern/unipose')

In [3]:
from groundingdino.util.inference import load_model
from SummerIntern.unipose import unipose_to_json as unipose

# Ignore warnings
import warnings
import sys
warnings.filterwarnings("ignore")

# Global variables
UNIPOSE_MODEL = unipose.load_model()
DINO_MODEL = load_model("groundingdino/config/GroundingDINO_SwinT_OGC.py", "weights/groundingdino_swint_ogc.pth")
UNIPOSE_CONFIG = {'box_threshold': 0.1, 'iou_threshold': 0.9}

OUTPUT_PATH = "../coco_dataset/test_grounding_dino"
INPUT_PATH = "../coco_dataset/imgs_a"
TEXT_PROMPT = "character . "
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25


final text_encoder_type: bert-base-uncased


In [5]:
# List and load images

import os
from glob import glob

def list_images(dir_path):
    # List all files in directory recursively
    return [y for x in os.walk(dir_path) for y in glob(os.path.join(x[0], '*.jpg'))] + \
           [y for x in os.walk(dir_path) for y in glob(os.path.join(x[0], '*.jpeg'))] + \
           [y for x in os.walk(dir_path) for y in glob(os.path.join(x[0], '*.png'))]

from groundingdino.util.inference import load_image

In [6]:
from groundingdino.util.inference import predict

# use grounding dino to extract boxes coordinates
def detect_character(image):
    boxes, logits, phrases = predict(
        model=DINO_MODEL,
        image=image,
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    return boxes, logits, phrases



In [7]:
import torch
from torchvision.ops.boxes import box_convert
import numpy as np

def process_boxes(boxes, image_source):
    h, w, _ = image_source.shape
    boxes = boxes * torch.Tensor([w, h, w, h])
    xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").numpy()
    xyxy = [list(map(int, sublist)) for sublist in xyxy.tolist()]
    return xyxy

In [8]:
import cv2
def crop_box(image, box):
    cropped = image[box[1]:box[3], box[0]:box[2], :]
    return cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)

def crop_boxes(image, boxes):
    return [crop_box(image, box) for box in boxes]

In [9]:



def predict_unipose(image, image_source):
    keypoints, _ = unipose.detect_keypoints(image, UNIPOSE_MODEL, UNIPOSE_CONFIG)
    keypoints, _, _ = unipose.process_keypoints(keypoints, image_source)
    return keypoints






In [10]:
from groundingdino.util.inference import annotate
import cv2
from PIL import ImageDraw
 
r = 5
image_source, image = load_image(".asset/cat_dog.jpeg")
boxes, logits, phrases = detect_character(image)
 
skeleton = [
        [1, 2], [2, 9], [9, 10], [10, 11], [10, 14], [11, 12], [12, 13], [14, 15], [15, 16], [2, 3], [3, 4], [4, 5], [2, 6], [6, 7], [7, 8]
    ]
 
for box in boxes: 
    # get the cropped image
    cropped = crop_box(image, box)
 
    # get the keypoints
    keypoints = predict_unipose(cropped, image)
 
    # annotate keypoints
    draw = ImageDraw.Draw(cropped)
    colors = []
    for i, keypoint in enumerate(keypoints): 
        draw.ellipse([keypoint[0]-r, keypoint[1]-r, keypoint[0]+r, keypoint[1]+r], fill=colors[i])
 
    for i, j in skeleton:
        draw.line([keypoints[i + 1][0], keypoints[i + 1][1], keypoints[j + 1][0], keypoints[j + 1][1]], fill="red", width=2)
 
    # paste the cropped image back to the original image
    image.paste(cropped, box)
 
 
cv2.imwrite("output.jpg", cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)) 

NameError: name '_C' is not defined